#Подключаем библиотеки

In [ ]:
!pip install -q pymorphy2 

     |████████████████████████████████| 55 kB 2.0 MB/s 
     |████████████████████████████████| 8.2 MB 302 kB/s 


In [ ]:
!pip install tensorflow==2.3 

In [ ]:
import numpy as np
from tensorflow.keras.models import Model, load_model 
import re 
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.layers import Dense, Embedding, Input, concatenate, Activation, MaxPooling1D, Conv1D, BatchNormalization, Dropout, Conv2DTranspose, Conv1DTranspose, Lambda
from tensorflow.keras import backend as K 
from tensorflow.keras.optimizers import Adam, Adadelta 
from tensorflow.keras import utils 
import tensorflow
from google.colab import files 
import matplotlib.pyplot as plt 
from gensim.models import word2vec
import os 
import pandas as pd
import time 
import nltk
from nltk.stem import WordNetLemmatizer 
import pymorphy2

nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
directory = '/content/drive/My Drive/Тексты/Решения/'

# Подготовка дата-сета

In [ ]:
def readText():
  agreements = [] 
  for filename in os.listdir(directory): 
    a=directory+filename
    f = open(a, 'r')
    text = f.read() 
    if text != '': 
      delSymbols = ['\n', "\t", "\ufeff", ".", "_", "-", ",", "!", "?", "–", "(", ")", "«", "»", "№", ";",'•','%'] # Определяем, какие знаки будут удалены
      for dS in delSymbols: 
        text = text.replace(dS, " ") 
        text = re.sub("[.]", " ", text) # re - для работы со строками
        text = re.sub(":", " ", text)
        text = re.sub("<", " <", text)
        text = re.sub(">", "> ", text)
        text = ' '.join(text.split()) 
        text = text.lower() # Конвертируем текст к нижнему регистру
      agreements.append(text)
  return agreements 


def text2Words(): 
  docs_full = [] 
  for i in range(len(readText())): 
    text=readText()[i]
    words = text.split(' ') 
    morph = pymorphy2.MorphAnalyzer() 
    docs = [morph.parse(word)[0].normal_form for word in words] 
    docs_full.append(docs)
  return docs_full

# Токенизация

In [ ]:
def getXYSamples(tok_agreem, tags_index):   
  tags01 = [] 
  indices = []
  for agreement in tok_agreem: 
    tag_place = [0, 0, 0, 0, 0, 0] 
    for ex in agreement: 
        if ex in tags_index: 
          place = np.argwhere(tags_index==ex) 
          if len(place)!=0: 
            if place[0][0]<6:
              tag_place[place[0][0]] = 1    
            else: 
              tag_place[place[0][0] - 6] = 0  
        else:          
          tags01.append(tag_place.copy()) 
          indices.append(ex) 
  return indices, tags01

def reverseIndex(clean_voc, x): 
  reverse_word_map = dict(map(reversed, clean_voc.items())) 
  docs = [reverse_word_map.get(letter) for letter in x] 
  return docs 

# Разделение выборки

In [ ]:
xLen = 256 
step = 30 
embeddingSize = 300 

def getSetFromIndices(wordIndices, xLen, step):  
  xBatch = [] 
  wordsLen = len(wordIndices) 
  index = 0   
  while (index + xLen <= wordsLen): 
    xBatch.append(wordIndices[index:index+xLen]) 
    index += step 

  return xBatch 

# Создание Word2Vec надстройки



In [ ]:
def pad_zeros(phrase, xLen = 256): 
  while len(phrase) < xLen: 
    phrase.append([0] * embeddingSize) 
  if len(phrase) > xLen: 
    phrase = phrase[:xLen] 
  return phrase 


def getSets(model, senI, tagI):
  xVector = [] 
  tmp = [] 
  for text in senI: 
    tmp=[]
    for word in text: 
      try: 
        tmp.append(model[word])
      except: 
        pass
    xVector.append(pad_zeros(tmp, xLen))
  temp = np.array(xVector)
  return np.array(xVector, dtype = np.float32), np.array(tagI)

# Создание модели PSPnet

In [ ]:
def dice_coef(y_true, y_pred):
    return (2. * K.sum(y_true * y_pred) + 1.) / (K.sum(y_true) + K.sum(y_pred) + 1.) 

In [ ]:
def create_PSPNet(conv_size = 64, num_classes = 6, input_shape = (30, 300)):
    img_input = Input(input_shape)

    x = Conv1D(conv_size, 3, padding='same')(img_input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = Conv1D(conv_size, 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x_mp_2 = MaxPooling1D(2)(x)
    x_mp_4 = MaxPooling1D(4)(x)
    x_mp_8 = MaxPooling1D(8)(x)
    x_mp_16 = MaxPooling1D(16)(x)
    x_mp_32 = MaxPooling1D(32)(x)

    x_mp_2 = Conv1D(conv_size, 3, padding='same', activation='relu')(x_mp_2)
    x_mp_2 = Dropout(0.5)(x_mp_2)

    x_mp_4 = Conv1D(conv_size, 3, padding='same', activation='relu')(x_mp_4)
    x_mp_4 = Dropout(0.5)(x_mp_4)

    x_mp_8 = Conv1D(conv_size, 3, padding='same', activation='relu')(x_mp_8)
    x_mp_8 = Dropout(0.5)(x_mp_8)

    x_mp_16 = Conv1D(conv_size, 3, padding='same', activation='relu')(x_mp_16)
    x_mp_16 = Dropout(0.5)(x_mp_16)

    x_mp_32 = Conv1D(conv_size, 3, padding='same', activation='relu')(x_mp_32)
    x_mp_32 = Dropout(0.5)(x_mp_32) 

    x_mp_2 = Conv1DTranspose(conv_size, 2, strides=2)(x_mp_2)
    x_mp_2 = Activation('relu')(x_mp_2)

    x_mp_4 = Conv1DTranspose(conv_size, 4, strides=4)(x_mp_4)
    x_mp_4 = Activation('relu')(x_mp_4)

    x_mp_8 = Conv1DTranspose(conv_size, 8, strides=8)(x_mp_8)
    x_mp_8 = Activation('relu')(x_mp_8)

    x_mp_16 = Conv1DTranspose(conv_size, 16, strides=16)(x_mp_16)
    x_mp_16 = Activation('relu')(x_mp_16)

    x_mp_32 = Conv1DTranspose(conv_size, 32, strides=32)(x_mp_32)
    x_mp_32 = Activation('relu')(x_mp_32)

    fin = concatenate([img_input, x_mp_2, x_mp_4, x_mp_8, x_mp_16, x_mp_32])    
    fin = Conv1D(conv_size, 3, padding='same')(fin)
    fin = BatchNormalization()(fin)
    fin = Activation('relu')(fin)
    fin = Conv1D(conv_size, 3, padding='same')(fin)
    fin = BatchNormalization()(fin)
    fin = Activation('relu')(fin)

    fin = Conv1D(num_classes, 3, activation='sigmoid', padding='same')(fin)

    model = Model(img_input, fin)
    model.compile(optimizer=Adadelta(lr=0.001),
                  loss='categorical_crossentropy',
                  metrics=[dice_coef])
    
    return model

# Обучение сети

In [ ]:
docs_full=text2Words()
tokenizer = Tokenizer(lower=True, filters = '', char_level=False) 
tokenizer.fit_on_texts(docs_full) 
clean_voc = {}
for item in tokenizer.word_index.items(): 
  clean_voc[item[0]] = item[1]  
doc = docs_full[0:-10] 
tok_agreem = tokenizer.texts_to_sequences(doc) 
docsToTest = docs_full[-10:] 
tok_agreemTest = tokenizer.texts_to_sequences(docsToTest) 

In [ ]:
tags_index = ['<s' + str(i) + '>' for i in range(1, 7)] 
closetags = ['</s' + str(i) + '>' for i in range(1, 7)] 
tags_index.extend(closetags) 
tags_index = np.array([clean_voc[i] for i in tags_index]) 

xData, yData = getXYSamples(tok_agreem,tags_index) 
decoded_text = reverseIndex(clean_voc, xData) 
xDataTest, yDataTest = getXYSamples(tok_agreemTest,tags_index) 
decoded_text_test = reverseIndex(clean_voc, xDataTest) 

In [ ]:
#xTrain = getSetFromIndices(decoded_text, xLen, step) 
#yTrain = getSetFromIndices(yData, xLen, step) 
#xTest = getSetFromIndices(decoded_text_test, xLen, step) 
#yTest = getSetFromIndices(yDataTest, xLen, step) 

In [ ]:
#modelGENSIM = word2vec.Word2Vec(xTrain + xTest, size = embeddingSize, window = 10, min_count = 1, workers = 10, iter = 10, max_vocab_size = 1e5)
#GENSIMtrainX, GENSIMtrainY = getSets(modelGENSIM, xTrain, yTrain)
#GENSIMtestX, GENSIMtestY = getSets(modelGENSIM, xTest, yTest)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  app.launch_new_instance()


In [ ]:
#model_b_PSPnet = create_PSPNet(input_shape=(xLen, embeddingSize))
#model_b_PSPnet.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 300)]   0                                            
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 256, 64)      57664       input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 256, 64)      256         conv1d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 256, 64)      0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [ ]:
history = model_b_PSPnet.fit(GENSIMtrainX, GENSIMtrainY, validation_data = (GENSIMtestX, GENSIMtestY), epochs=1000, batch_size=64)

Epoch 1/1000
13/13 [==============================] - 1s 76ms/step - loss: 0.2979 - dice_coef: 0.0872 - val_loss: 0.0520 - val_dice_coef: 0.0129
Epoch 2/1000
13/13 [==============================] - 1s 67ms/step - loss: 0.2977 - dice_coef: 0.0878 - val_loss: 0.0520 - val_dice_coef: 0.0129
Epoch 3/1000
13/13 [==============================] - 1s 67ms/step - loss: 0.2978 - dice_coef: 0.0938 - val_loss: 0.0520 - val_dice_coef: 0.0129
Epoch 4/1000
13/13 [==============================] - 1s 66ms/step - loss: 0.2974 - dice_coef: 0.0904 - val_loss: 0.0520 - val_dice_coef: 0.0129
Epoch 5/1000
13/13 [==============================] - 1s 66ms/step - loss: 0.2977 - dice_coef: 0.0959 - val_loss: 0.0520 - val_dice_coef: 0.0129
Epoch 6/1000
13/13 [==============================] - 1s 64ms/step - loss: 0.2975 - dice_coef: 0.0900 - val_loss: 0.0520 - val_dice_coef: 0.0129
Epoch 7/1000
13/13 [==============================] - 1s 65ms/step - loss: 0.2970 - dice_coef: 0.0937 - val_loss: 0.0519 - val_dic

# Результаты сети

In [ ]:
def recognizeSet(XX, YY, model, tags, length, value):
  correct_list = np.array([0] * 6) 
  incorrect_list =  np.array([0] * 6)  
  XX_array = XX
  YY_array = YY
  pred = model.predict(XX_array)
  pred[pred < value] = 0
  pred[pred > value] = 1
  for element in range(YY_array.shape[0]): 
    for word in range(YY_array.shape[1]):  
      for category in range(YY_array.shape[2]): 
        if pred[element][word][category] == YY_array[element][word][category]: 
          correct_list[category] += 1 
        else:  
          incorrect_list[category] += 1       
  for i in range(6): 
   print("Сеть распознала категорию  '{}' с точностью в {}%".format(tags[i], round(100*correct_list[i]/(correct_list[i] + incorrect_list[i]), 2)))
  total = round(100*np.mean(correct_list/(correct_list + incorrect_list)),2) 
  print("Средняя точность {}%".format(total))

In [ ]:
tags = ['S1', 'S2', 'S3', 'S4', 'S5', 'S6'] # <s1> - номер дела, <s2> - дата решения, <s3> - имя судьи, <s4> - истец, <s5> - суть искового заявления, <s6> - ответчик
recognizeSet(GENSIMtestX, GENSIMtestY, model_b_PSPnet, tags, xLen, 0.5) 

Сеть распознала категорию  'S1' с точностью в 97.55%
Сеть распознала категорию  'S2' с точностью в 9.39%
Сеть распознала категорию  'S3' с точностью в 99.89%
Сеть распознала категорию  'S4' с точностью в 45.5%
Сеть распознала категорию  'S5' с точностью в 67.92%
Сеть распознала категорию  'S6' с точностью в 54.32%
Средняя точность 62.43%
